In [ ]:
import numpy as np

generate EMI convolution matrix E

valid padding
the height of E is (kx-a) * (ky-b), a = (delta_kx - 1)/2, b = (delta_ky - 1)/2
the width is similar


In [17]:
def EMI_conv_matrix(detectors, window_size):
    delta_kx, delta_ky = window_size
    lines = []
    kx, ky = detectors[0].shape
    for i_x in range(kx - delta_kx + 1):
        for i_y in range(ky - delta_ky + 1):
            line = np.concatenate(tuple(
                d[i_x:i_x+delta_kx, i_y:i_y+delta_ky].flatten() for d in detectors
            ))
            lines.append(line)
    return np.vstack(tuple(lines))

detectors = np.array([
    [[111, 112, 113],
     [121, 122, 123],
     [131, 132, 133],
     [141, 142, 143]],
    [[211, 212, 213],
     [221, 222, 223],
     [231, 232, 233],
     [241, 242, 243]]
])

window_size = 3, 1
ecm = EMI_conv_matrix(detectors, window_size)
print(ecm.shape)
print(ecm)

(6, 6)
[[111 121 131 211 221 231]
 [112 122 132 212 222 232]
 [113 123 133 213 223 233]
 [121 131 141 221 231 241]
 [122 132 142 222 232 242]
 [123 133 143 223 233 243]]


calculate h_k

In [9]:
def calculate_h(E, e):
    h = np.linalg.pinv(E) @ e
    assert h.shape[0] == 1
    return h


ValueError: need at least one array to concatenate

In [ ]:
def divide_groups(E, e, W):
    width = E.shape[1]
    if width % W != 0:
        print("kx 必须是 W 的整数倍！")
        return

    sub_width = width / W
    return [[E[:, k*sub_width:(k+1)*sub_width], e[:, k*sub_width:(k+1)*sub_width]] for k in range(W)]


In [ ]:
def cluster(H, threshold=0.5):

    # normalize
    H = H / np.linalg.norm(H, axis=0)

    H_conj_T = H.conj().T
    C = np.dot(H, H_conj_T)
    C_threshold = (C >= threshold).astype(int)

    group_bound = [0]
    W = H.shape[0]
    y = 0
    while y < W:
        x = y
        while C_threshold[x, y] == 1:
            y += 1
        group_bound.append(y)
    return group_bound


In [ ]:
def EDITER(E, e, W, ):
    groups = divide_groups(E, e, W)

    H = [calculate_h(E, e)
        for E, e in groups]

    clustered_bounds = cluster(H)
    assert clustered_bounds[-1] == W
    bounds_pairs = [[clustered_bounds[i], clustered_bounds[i+1]] for i in range(len(clustered_bounds))]
    clustered_groups = [[E[:, i:j], e[:, i:j]] for i, j in bounds_pairs]
    clustered_H = [calculate_h(E, e)
        for E, e in clustered_groups]
    return H, clustered_bounds[1:]
